In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from timeit import timeit

In [ ]:
setup = """
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_csv('C:/Users/benig/Desktop/arcos_data_3d.csv')
df.dropna(inplace=True)
"""

In [ ]:
def Normalize(data):
    return (data - np.min(data))/(np.max(data)- np.min(data))

In [ ]:
df_grp = df.groupby(['collid', 'trackID'])
colors = plt.cm.viridis(np.linspace(0,1, 10))

In [ ]:
mycode = """
df_grp = df.groupby(['collid'])
df['groups'] = df_grp.ngroup()
df_grp = df.groupby(['groups', 'trackID'])
colors = plt.cm.jet(np.linspace(0,1, 2))
projection = 'posx'
for i, dat in df_grp:
    plt.plot(dat['time'], dat[projection], c=colors[int(i[0])])
plt.show()
plt.close()
"""

timeit(stmt=mycode, setup=setup, number=10)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
mycode = """
df.sort_values(["collid", 'trackID'], inplace=True)
array = df[['collid', 'trackID', 'time', 'posx', 'posy', 'posz']].to_numpy()
grouped_array = np.split(
    array, np.unique(array[:, 0], axis=0, return_index=True)[1][1:]
)
seq_colids = np.concatenate([np.repeat(i, value.shape[0]) for i,value in enumerate(grouped_array)], axis=0)
array_seq_colids = np.column_stack((array, seq_colids))
grouped_array = np.split(
    array_seq_colids, np.unique(array_seq_colids[:, :2], axis=0, return_index=True)[1][1:]
)
colors = plt.cm.viridis_r(np.linspace(0,1, np.unique(array_seq_colids[:,-1]).shape[0]))

for dat in grouped_array:
    plt.plot(dat[:,2], dat[:,5], c=colors[int(dat[0,-1])])

plt.show()
plt.close()
"""

timeit(stmt=mycode, setup=setup, number=10)

In [ ]:
plt.cm.turbo

In [ ]:
a = grouped_array_2.copy()
for i, value in enumerate(grouped_array_2):
    np.concatenate([a,np.repeat(i, value.size)])

np.

In [ ]:
out.reshape(-1,1)

In [1]:
import napari
from scipy import ndimage
from skimage import io

viewer = napari.Viewer()

In [2]:

nuclei = io.imread("C:/Users/benig/Documents/python_dev/arcos-gui/arcos-gui/src/arcos_gui/nuc_transposed.tiff")


In [ ]:
erkktr = io.imread("C:/Users/benig/Desktop/C2-20191021_WT_control.tif")

In [ ]:
nuclei.shape

In [ ]:
nuclei_zoomed = ndimage.zoom(nuclei, (1,0.6,0.37,0.37))

In [ ]:
erktr_zoomed = ndimage.zoom(erkktr, (1,0.6,0.37,0.37)) 

In [3]:
viewer.add_image(nuclei)

<Image layer 'nuclei' at 0x2539eed74c0>

c:\users\benig\documents\python_dev\arcos-gui\arcos-gui\src\arcos_gui\_arcos_widgets.py:886: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  clid_np_sorted = clid_np[[clids_sorted_i]]
C:\Users\benig\miniconda3\envs\arcos\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
C:\Users\benig\miniconda3\envs\arcos\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
C:\Users\benig\miniconda3\envs\arcos\lib\site-packages\numpy\core\numeric.py:2449: FutureWarni

In [ ]:
import pandas as pd

data = pd.read_csv("C:/Users/benig/Desktop/tracks_191021_wt_curated_origSmoothedXYZ_interpolated_binarised.csv.gz")

In [ ]:
viewer = napari.Viewer()

In [ ]:
import numpy as np

In [ ]:
img_transposed = np.transpose(nuclei_zoomed, [0,2,3,1])

In [ ]:
io.imsave("nuc_transposed.tiff", img_transposed)